# Trabalho de Geração Termelétrica e Cogeração
## 2023.3
### 15 de Dezembro de 2023

Componentes do grupo:
- Felipe Bogaerts de Mattos
- Igor Lopes Passi
- Rafael Malta

## Import modules


In [1]:
import numpy as np
import plotly.graph_objects as go
from pyfluids import Fluid, FluidsList, Input

from thermosys.processes import (
    heat_to_temperature,
    turbine_to_enthalpy,
    turbine_to_pressure,
)
from thermosys.processes.gas import compress_to
from thermosys.processes.vapor import condense_to_pressure, pump_to_pressure
from thermosys.services.energy import energy_balance
from thermosys.services.misc import print_states
from thermosys.services.montecarlo import get_random_value
from thermosys.services.plots import plot_saturation_curve, add_isobaric_process_to_plot, mark_state_ts, plot_isobaric_process, plot_expansion_process, plot_compression_process, plot_isoenthalpic_process
from thermosys.services.units import bar_to_pascal, pascal_to_bar

## Constants


In [2]:
GAS_MASS_FLOW_PER_SYSTEM = 450  # kg/s
NUMBER_OF_GAS_SYSTEMS = 2  # -

GAS_INLET_PRESSURE = bar_to_pascal(1)  # Pa
GAS_INLET_TEMPERATURE = 30  # C
GAS_MASS_FLUX = 450  # kg/s
GAS_COMPRESSOR_EFFICIENCY = 0.8  # -
GAS_COMPRESSION_RATIO = 25  # -
GAS_TURBINE_C_EFFICIENCY = 0.8  # -
GAS_COMBUSTION_CHAMBER_1_TEMPERATURE = 1000  # C
GAS_TURBINE_P_EFFICIENCY = 0.85  # -
GAS_COMBUSTION_CHAMBER_2_TEMPERATURE = 710  # C

VAPOR_DEAERATOR_PRESSURE = bar_to_pascal(5)  # Pa
VAPOR_TURBINE_EFFICIENCY = 0.9  # -
VAPOR_CONDENSER_PRESSURE = bar_to_pascal(0.1)  # Pa
VAPOR_PUMP_EFFICIENCY = 0.9999  # -
VAPOR_DEAERATOR_TEMPERATURE = 120  # C
VAPOR_STATE_3_QUALITY_LOWER_THRESHOLD = 0.92  # -

AMBIENT_PRESSURE = bar_to_pascal(1)  # Pa
AMBIENT_TEMPERATURE = 30  # C

RECOVERY_BOILER_DELTA_TSA = 25  # C
RECOVERY_BOILER_DELTA_TP = 10  # C
RECOVERY_BOILER_DELTA_TE = 10  # C

OPTIMIZATION_ITERATION_COUNT = 1000  # -
MAX_VAPOR_INLET_PRESSURE = bar_to_pascal(100)  # Pa

## Brayton Gas Cycle


In [3]:
state_1 = Fluid(FluidsList.Air).with_state(
    Input.pressure(GAS_INLET_PRESSURE),
    Input.temperature(GAS_INLET_TEMPERATURE),
)

state_2 = compress_to(
    inlet_state=state_1,
    efficiency=GAS_COMPRESSOR_EFFICIENCY,
    compression_ratio=GAS_COMPRESSION_RATIO,
)

state_3 = heat_to_temperature(
    inlet_state=state_2,
    outlet_temperature=GAS_COMBUSTION_CHAMBER_1_TEMPERATURE,
)

energy_balance_compressor = energy_balance(state_1, state_2)
state_4 = turbine_to_enthalpy(
    inlet_state=state_3,
    efficiency=GAS_TURBINE_C_EFFICIENCY,
    energy_balance=energy_balance_compressor,
)

state_5 = heat_to_temperature(
    inlet_state=state_4,
    outlet_temperature=GAS_COMBUSTION_CHAMBER_2_TEMPERATURE,
)

state_6 = turbine_to_pressure(
    inlet_state=state_5,
    efficiency=GAS_TURBINE_P_EFFICIENCY,
    outlet_pressure=GAS_INLET_PRESSURE,
)

gas_states = [state_1, state_2, state_3, state_4, state_5, state_6]

gas_turbine_c_work = (
    energy_balance(gas_states[3], gas_states[2]) * GAS_MASS_FLOW_PER_SYSTEM
)
gas_turbine_p_work = (
    energy_balance(gas_states[5], gas_states[4]) * GAS_MASS_FLOW_PER_SYSTEM
)
gas_compressor_work = (
    energy_balance(gas_states[0], gas_states[1]) * GAS_MASS_FLOW_PER_SYSTEM
)
gas_net_work = gas_turbine_c_work + gas_turbine_p_work - gas_compressor_work
gas_heat_source_1 = (
    energy_balance(gas_states[1], gas_states[2]) * GAS_MASS_FLOW_PER_SYSTEM
)
gas_heat_source_2 = (
    energy_balance(gas_states[3], gas_states[4]) * GAS_MASS_FLOW_PER_SYSTEM
)
gas_heat_in = gas_heat_source_1 + gas_heat_source_2

gas_efficiency = gas_net_work / gas_heat_in
gas_bwr = gas_compressor_work / (gas_turbine_c_work + gas_turbine_p_work)

print("BRAYTON CYCLE")
print_states(gas_states)


print("\n")

print(f"Gas turbine TC work: {gas_turbine_c_work * 1e-6:.2f} MW")
print(f"Gas compressor specific work: {gas_compressor_work * 1e-6:.2f} MW")
print(f"Gas net specific work: {gas_net_work * 1e-6:.2f} MW")
print(f"Gas heat source 1: {gas_heat_source_1 * 1e-6:.2f} MW")
print(f"Gas heat source 1: {gas_heat_source_2 * 1e-6:.2f} MW")
print(f"Gas heat in: {gas_heat_in * 1e-6:.2f} MW")
print(f"Gas thermal efficiency: {gas_efficiency * 100:.2f} %")
print(f"Gas BWR: {gas_bwr * 100:.2f} %")

print("\n")

BRAYTON CYCLE
1 - Air: 1.00 bar, 30.00 C, 429.47 kJ/kg
2 - Air: 25.00 bar, 573.80 C, 1001.22 kJ/kg
3 - Air: 25.00 bar, 1000.00 C, 1492.34 kJ/kg
4 - Air: 1.72 bar, 501.44 C, 920.59 kJ/kg
5 - Air: 1.72 bar, 710.00 C, 1153.36 kJ/kg
6 - Air: 1.00 bar, 602.03 C, 1031.65 kJ/kg


Gas turbine TC work: 257.29 MW
Gas compressor specific work: 257.29 MW
Gas net specific work: 54.77 MW
Gas heat source 1: 221.00 MW
Gas heat source 1: 104.75 MW
Gas heat in: 325.75 MW
Gas thermal efficiency: 16.81 %
Gas BWR: 82.45 %




# Optimization of the Rankine Vapor Cycle and Recovery Boiler


In [4]:
pressures = np.array([])
system_efficiencies = np.array([])

vapor_states_list = []
gas_states_in_boiler_list = []
boiler_states_list = []

# for _ in range(OPTIMIZATION_ITERATION_COUNT):
for _ in range(1):
    # Optimization variables:
    inlet_pressure = get_random_value(
        min_value=VAPOR_DEAERATOR_PRESSURE, max_value=MAX_VAPOR_INLET_PRESSURE
    )

    inlet_pressure = bar_to_pascal(10)

    # Solving the Rankine Cycle:
    state_7 = Fluid(FluidsList.Water).with_state(
        Input.pressure(inlet_pressure),
        Input.temperature(VAPOR_DEAERATOR_TEMPERATURE),
    )

    state_1 = heat_to_temperature(
        inlet_state=state_7,
        outlet_temperature=gas_states[-1].temperature
        - RECOVERY_BOILER_DELTA_TSA,
    )

    state_2 = turbine_to_pressure(
        inlet_state=state_1,
        efficiency=VAPOR_TURBINE_EFFICIENCY,
        outlet_pressure=VAPOR_DEAERATOR_PRESSURE,
    )

    state_3 = turbine_to_pressure(
        inlet_state=state_2,
        efficiency=VAPOR_TURBINE_EFFICIENCY,
        outlet_pressure=VAPOR_CONDENSER_PRESSURE,
    )

    state_4 = condense_to_pressure(
        inlet_state=state_3,
    )

    state_5 = pump_to_pressure(
        inlet_state=state_4,
        efficiency=VAPOR_PUMP_EFFICIENCY,
        outlet_pressure=VAPOR_DEAERATOR_PRESSURE,
    )

    state_6 = state_5.with_state(
        Input.pressure(state_5.pressure),
        Input.entropy(state_7.entropy),
    )

    vapor_states = [
        state_1,
        state_2,
        state_3,
        state_4,
        state_5,
        state_6,
        state_7,
    ]

    y_mass_flow = energy_balance(state_6, state_5) / energy_balance(
        state_2, state_5
    )

    vapor_turbine_specific_work = energy_balance(
        vapor_states[1], vapor_states[0]
    ) + (1 - y_mass_flow) * energy_balance(vapor_states[2], vapor_states[1])
    vapor_pump_specific_work = (1 - y_mass_flow) * energy_balance(
        vapor_states[3], vapor_states[4]
    ) + energy_balance(vapor_states[5], vapor_states[6])
    vapor_net_specific_work = (
        vapor_turbine_specific_work - vapor_pump_specific_work
    )
    vapor_specific_heat_in = energy_balance(vapor_states[-1], vapor_states[0])

    vapor_efficiency = vapor_net_specific_work / vapor_specific_heat_in

    # Solving the Recovery Boiler:
    gas_mass_flow = GAS_MASS_FLOW_PER_SYSTEM * NUMBER_OF_GAS_SYSTEMS

    state_8_evaporator = Fluid(FluidsList.Water).two_phase_point_at_pressure(
        pressure=vapor_states[0].pressure,
        quality=0,
    )

    state_12 = Fluid(FluidsList.Water).two_phase_point_at_pressure(
        pressure=vapor_states[0].pressure,
        quality=100,
    )

    temperature_a = state_8_evaporator.temperature + RECOVERY_BOILER_DELTA_TP
    state_a = Fluid(FluidsList.Air).with_state(
        Input.pressure(gas_states[-1].pressure),
        Input.temperature(temperature_a),
    )

    temperature_8_economizer = (
        state_8_evaporator.temperature - RECOVERY_BOILER_DELTA_TE
    )
    state_8_economizer = Fluid(FluidsList.Water).with_state(
        Input.pressure(vapor_states[0].pressure),
        Input.temperature(temperature_8_economizer),
    )

    heat_evaporator_sup = (
        energy_balance(state_a, gas_states[-1]) * gas_mass_flow
    )

    mass_flow_evaporator = heat_evaporator_sup / (
        energy_balance(vapor_states[0], state_8_evaporator)
    )

    enthalpy_b = gas_states[
        -1
    ].enthalpy - mass_flow_evaporator / gas_mass_flow * (
        energy_balance(state_8_economizer, vapor_states[-1])
    )

    state_b = state_a.with_state(
        Input.enthalpy(enthalpy_b), Input.pressure(gas_states[-1].pressure)
    )

    outlet_gas_enthalpy = (
        state_a.enthalpy
        - mass_flow_evaporator
        / gas_mass_flow
        * (energy_balance(state_8_economizer, vapor_states[-1]))
    )

    outlet_gas_state = state_b.with_state(
        Input.pressure(gas_states[-1].pressure),
        Input.temperature(state_b.temperature),
    )

    state_ambient = Fluid(FluidsList.Air).with_state(
        Input.pressure(AMBIENT_PRESSURE),
        Input.temperature(AMBIENT_TEMPERATURE),
    )

    enthalpy_7_gas = state_a.enthalpy

    gas_states_in_boiler = [
        state_a,
        state_b,
        state_7,
    ]

    boiler_states = [
        state_8_economizer,
        state_8_evaporator,
        state_12,
    ]

    gas_heat_superheater = mass_flow_evaporator * energy_balance(
        state_8_evaporator, vapor_states[-1]
    )
    gas_heat_evaporator = mass_flow_evaporator * energy_balance(
        state_b, state_a
    )
    gas_heat_economizer = mass_flow_evaporator * energy_balance(
        state_a, outlet_gas_state
    )

    vapor_heat_evaporator = mass_flow_evaporator * energy_balance(
        state_12, state_8_evaporator
    )

    boiler_efficiency = energy_balance(
        gas_states[-1], outlet_gas_state
    ) / energy_balance(gas_states[-1], state_ambient)

    pressures = np.append(pressures, inlet_pressure)
    vapor_states_list.append(vapor_states)
    gas_states_in_boiler_list.append(gas_states_in_boiler)
    boiler_states_list.append(boiler_states)
    system_efficiencies = np.append(
        system_efficiencies,
        gas_efficiency
        + vapor_efficiency * boiler_efficiency * (1 - gas_efficiency),
    )

## Processing the simulation data

After running the simulation, the pressure and efficiency arrays are filtered so that the quality of the vapor state 3 is always higher than 92% - as specified in the requirements.

In [5]:
valid_indices = np.array([], dtype=int)

for i, states in enumerate(vapor_states_list):
    state_3 = states[2]

    if state_3.phase.name == "TwoPhase":
        if state_3.quality >= VAPOR_STATE_3_QUALITY_LOWER_THRESHOLD * 100:
            valid_indices = np.append(valid_indices, i)
    elif state_3.phase.name == "Gas":
        valid_indices = np.append(valid_indices, i)

print(
    f"Filtered {len(vapor_states_list) - np.shape(valid_indices)[0]} invalid states out of {len(vapor_states_list)}."
)

# Use these indices to filter the pressures and system_efficiencies:
pressures = pressures[valid_indices]
system_efficiencies = system_efficiencies[valid_indices]

vapor_states_list[:] = [vapor_states_list[i] for i in valid_indices]
gas_states_in_boiler_list[:] = [gas_states_in_boiler_list[i] for i in valid_indices]
boiler_states_list[:] = [boiler_states_list[i] for i in valid_indices]

max_efficiency = np.max(system_efficiencies)
max_efficiency_index = np.argmax(system_efficiencies)
optimal_inlet_pressure = pressures[max_efficiency_index]
optimal_vapor_states = vapor_states_list[max_efficiency_index]
optimal_gas_states_in_boiler = gas_states_in_boiler_list[max_efficiency_index]
optimal_boiler_states = boiler_states_list[max_efficiency_index]

Filtered 0 invalid states out of 1.


## Optimal Rankine Cycle and Boiler results

In [6]:
optimal_vapor_state_1 = optimal_vapor_states[0]
optimal_vapor_state_2 = optimal_vapor_states[1]
optimal_vapor_state_3 = optimal_vapor_states[2]
optimal_vapor_state_4 = optimal_vapor_states[3]
optimal_vapor_state_5 = optimal_vapor_states[4]
optimal_vapor_state_6 = optimal_vapor_states[5]
optimal_vapor_state_7 = optimal_vapor_states[6]

y_mass_flow = energy_balance(
    optimal_vapor_state_6, optimal_vapor_state_5
) / energy_balance(optimal_vapor_state_2, optimal_vapor_state_5)

vapor_turbine_specific_work = energy_balance(
    optimal_vapor_state_2, optimal_vapor_state_1
) + (1 - y_mass_flow) * energy_balance(
    optimal_vapor_state_3, optimal_vapor_state_2
)
vapor_pump_specific_work = (1 - y_mass_flow) * energy_balance(
    optimal_vapor_state_4, optimal_vapor_state_5
) + energy_balance(optimal_vapor_state_6, optimal_vapor_state_5)
vapor_net_specific_work = (
    vapor_turbine_specific_work - vapor_pump_specific_work
)
vapor_specific_heat_in = energy_balance(
    optimal_vapor_state_7, optimal_vapor_state_1
)

vapor_efficiency = vapor_net_specific_work / vapor_specific_heat_in

print("OPTIMAL RANKINE CYCLE")
print_states(optimal_vapor_states)

print("\n")

print(f"Y mass flow: {y_mass_flow * 100:.2f} %")
print(
    f"Vapor turbine specific work: {vapor_turbine_specific_work * 1e-3:.2f} kJ/kg"
)
print(f"Vapor pump specific work: {vapor_pump_specific_work * 1e-3:.2f} kJ/kg")
print(f"Vapor net specific work: {vapor_net_specific_work * 1e-3:.2f} kJ/kg")
print(f"Vapor specific heat in: {vapor_specific_heat_in * 1e-3:.2f} kJ/kg")
print(f"Vapor thermal efficiency: {vapor_efficiency * 100:.2f} %")

optimal_gas_state_in_boiler_a = optimal_gas_states_in_boiler[0]
optimal_gas_state_in_boiler_b = optimal_gas_states_in_boiler[1]
optimal_gas_state_in_boiler_7 = optimal_gas_states_in_boiler[2]

print("\nBRAYTON CYCLE CONTINUATION")
print(
    f"a - {optimal_gas_state_in_boiler_a.name}: {optimal_gas_state_in_boiler_a.pressure * 1e-5:.2f} bar,"
    f" {optimal_gas_state_in_boiler_a.temperature:.2f} C, {optimal_gas_state_in_boiler_a.enthalpy * 1e-3:.2f} kJ/kg"
)
print(
    f"b - {optimal_gas_state_in_boiler_b.name}: {optimal_gas_state_in_boiler_b.pressure * 1e-5:.2f} bar,"
    f" {optimal_gas_state_in_boiler_b.temperature:.2f} C, {optimal_gas_state_in_boiler_b.enthalpy * 1e-3:.2f} kJ/kg"
)
print(
    f"7 - {optimal_gas_state_in_boiler_7.name}: {optimal_gas_state_in_boiler_7.pressure * 1e-5:.2f} bar,"
    f" {optimal_gas_state_in_boiler_7.temperature:.2f} C, {optimal_gas_state_in_boiler_7.enthalpy * 1e-3:.2f} kJ/kg"
)

optimal_boiler_state_8_economizer = optimal_boiler_states[0]
optimal_boiler_state_8_evaporator = optimal_boiler_states[1]
optimal_boiler_state_12 = optimal_boiler_states[2]

print("\nRECOVERY BOILER")
print(
    f"8 econ - {optimal_boiler_state_8_economizer.name}: {optimal_boiler_state_8_economizer.pressure * 1e-5:.2f} bar,"
    f" {optimal_boiler_state_8_economizer.temperature:.2f} C, {optimal_boiler_state_8_economizer.enthalpy * 1e-3:.2f} kJ/kg"
)
print(
    f"12 - {optimal_boiler_state_12.name}: {optimal_boiler_state_12.pressure * 1e-5:.2f} bar,"
    f" {optimal_boiler_state_12.temperature:.2f} C, {optimal_boiler_state_12.enthalpy * 1e-3:.2f} kJ/kg"
)
print(
    f"8 evap - {optimal_boiler_state_8_evaporator.name}: {optimal_boiler_state_8_evaporator.pressure * 1e-5:.2f} bar,"
    f" {optimal_boiler_state_8_evaporator.temperature:.2f} C, {optimal_boiler_state_8_evaporator.enthalpy * 1e-3:.2f} kJ/kg"
)

OPTIMAL RANKINE CYCLE
1 - Water: 10.00 bar, 577.03 C, 3647.66 kJ/kg
2 - Water: 5.00 bar, 470.51 C, 3421.34 kJ/kg
3 - Water: 0.10 bar, 68.06 C, 2626.58 kJ/kg
4 - Water: 0.10 bar, 45.81 C, 191.81 kJ/kg
5 - Water: 5.00 bar, 45.82 C, 192.30 kJ/kg
6 - Water: 5.00 bar, 119.96 C, 503.85 kJ/kg
7 - Water: 10.00 bar, 120.00 C, 504.38 kJ/kg


Y mass flow: 9.65 %
Vapor turbine specific work: 944.40 kJ/kg
Vapor pump specific work: 311.99 kJ/kg
Vapor net specific work: 632.41 kJ/kg
Vapor specific heat in: 3143.29 kJ/kg
Vapor thermal efficiency: 20.12 %

BRAYTON CYCLE CONTINUATION
a - Air: 1.00 bar, 189.88 C, 591.44 kJ/kg
b - Air: 1.00 bar, 572.64 C, 998.96 kJ/kg
7 - Water: 10.00 bar, 120.00 C, 504.38 kJ/kg

RECOVERY BOILER
8 econ - Water: 10.00 bar, 169.88 C, 718.66 kJ/kg
12 - Water: 10.00 bar, 179.88 C, 2777.11 kJ/kg
8 evap - Water: 10.00 bar, 179.88 C, 762.52 kJ/kg


# Plotting Monte Carlo results

In [7]:
print(f"Max efficiency: {max_efficiency * 100:.2f} %")
print(
    f"Vapor inlet pressure for max efficiency: {pascal_to_bar(optimal_inlet_pressure):.2f} bar"
)

fig = go.Figure(
    data=go.Scatter(
        x=pascal_to_bar(pressures),
        y=system_efficiencies * 100,
        mode="markers",
    )
)
fig.update_layout(
    title="Monte Carlo Simulation: Pressure vs Efficiency",
    xaxis_title="Pressure (bar)",
    yaxis_title="Efficiency (%)",
)
fig.show()

Max efficiency: 18.17 %
Vapor inlet pressure for max efficiency: 10.00 bar


## Plotting TS diagrams

In [8]:
fig = plot_saturation_curve(gas_states[-1])
add_isobaric_process_to_plot(
    gas_states[-1], gas_states[0].pressure, fig, line_style={"color": "red"}
)
add_isobaric_process_to_plot(
    gas_states[-1], gas_states[1].pressure, fig, line_style={"color": "green"}
)
add_isobaric_process_to_plot(
    gas_states[-1], gas_states[3].pressure, fig, line_style={"color": "blue"}
)

for state in gas_states:
    mark_state_ts(state, fig)

plot_compression_process(
    gas_states[0], gas_states[1], GAS_COMPRESSOR_EFFICIENCY, fig
)
plot_isobaric_process(gas_states[1], gas_states[2], fig)
plot_expansion_process(
    gas_states[2], gas_states[3], GAS_TURBINE_C_EFFICIENCY, fig
)
plot_isobaric_process(gas_states[3], gas_states[4], fig)
plot_expansion_process(
    gas_states[4], gas_states[5], GAS_TURBINE_P_EFFICIENCY, fig
)

fig.update_layout(yaxis_range=[gas_states[0].temperature - 20, gas_states[2].temperature + 200], xaxis_range=[3, 6])

fig.show()

In [9]:
fig = plot_saturation_curve(optimal_vapor_states[-1])
add_isobaric_process_to_plot(
    optimal_vapor_states[-1],
    optimal_vapor_states[0].pressure,
    fig,
    line_style={"color": "red"},
)
add_isobaric_process_to_plot(
    optimal_vapor_states[-1],
    optimal_vapor_states[1].pressure,
    fig,
    line_style={"color": "green"},
)

for state in optimal_vapor_states:
    mark_state_ts(state, fig)

plot_expansion_process(
    optimal_vapor_states[0],
    optimal_vapor_states[1],
    VAPOR_TURBINE_EFFICIENCY,
    fig,
)
plot_expansion_process(
    optimal_vapor_states[1],
    optimal_vapor_states[2],
    VAPOR_TURBINE_EFFICIENCY,
    fig,
)
plot_isobaric_process(optimal_vapor_states[2], optimal_vapor_states[3], fig)
plot_compression_process(optimal_vapor_states[3], optimal_vapor_states[4], VAPOR_PUMP_EFFICIENCY, fig)
plot_isobaric_process(optimal_vapor_states[4], optimal_vapor_states[5], fig)
plot_compression_process(optimal_vapor_states[5], optimal_vapor_states[6], VAPOR_PUMP_EFFICIENCY, fig)
plot_isobaric_process(optimal_vapor_states[6], optimal_vapor_states[0], fig)

fig.show()